# Cleaning Your Data

Let's take a web access log, and figure out the most-viewed pages on a website from it! Sounds easy, right?

Let's set up a regex that lets us parse an Apache access log line:

In [1]:
import re

format_pat= re.compile(
    r"(?P<host>[\d\.]+)\s"
    r"(?P<identity>\S*)\s"
    r"(?P<user>\S*)\s"
    r"\[(?P<time>.*?)\]\s"
    r'"(?P<request>.*?)"\s'
    r"(?P<status>\d+)\s"
    r"(?P<bytes>\S*)\s"
    r'"(?P<referer>.*?)"\s'
    r'"(?P<user_agent>.*?)"\s*'
)


Here's the path to the log file I'm analyzing:

In [10]:
logPath = "access_log.txt"

Now we'll whip up a little script to extract the URL in each access, and use a dictionary to count up the number of times each one appears. Then we'll sort it and print out the top 20 pages. What could go wrong?

In [12]:
URLCounts = {}  # Dicionario para armazenar a contagem de acessos a cada URL

with open(logPath, "r") as f:  # Abre o arquivo de logPath para leitura
    for line in (l.rstrip() for l in f):  # passa por cada linha do arquivo, removendo espaços em branco a direita
        match = format_pat.match(line)  # verifica se a linha corresponde ao padrao definido 
        if match:  # Se a linha corresponde ao padrão
            access = match.groupdict()  # extrai os grupos nomeados do padrao como um dicionario
            request = access['request']  # Obtem o campo 'request' do dicionário
            
            # Verifica se o formato do campo 'request' esta correto antes de desempacotar
            parts = request.split()  # Divide a string 'request' em partes separadas por espaço
            if len(parts) == 3:  # Verifica se ha exatamente 3 partes (metodo, URL, protocolo)
                action, URL, protocol = parts  # Desempacota as partes em variaveis
                # Atualiza a contagem de acessos para a URL
                if URL in URLCounts:  # Se a URL ja esta no dicionário
                    URLCounts[URL] = URLCounts[URL] + 1  # Incrementa a contagem
                else:  # Caso contrario
                    URLCounts[URL] = 1  # adiciona a URL com uma contagem inicial de 1
            else:  # Se o formato do 'request' não é esperado
                print(f"Formato inesperado: {request}")  # tratamento de erro com o request

# Ordena as URLs por contagem de acessos em ordem decrescente
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Exibe as 20 URLs mais acessadas com suas contagens
for result in results[:20]:  # Itera sobre as 20 primeiras URLs ordenadas
    print(result + ": " + str(URLCounts[result]))  # Exibe a URL e sua contagem de acessos


Formato inesperado: _\xb0ZP\x07tR\xe5
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
Formato inesperado: 
/xmlrpc.php: 68494
/wp-login.php: 1923
/: 440
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/business/: 70
/national/: 70
/national-headlines/: 70
/world/: 70
/weather/: 70


Hm. The 'request' part of the line is supposed to look something like this:

GET /blog/ HTTP/1.1

There should be an HTTP action, the URL, and the protocol. But it seems that's not always happening. Let's print out requests that don't contain three items:

In [28]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match = format_pat.match(line)
        if match:
            access = match.groupdict()
            request = access['request']
            fields = request.split()
            
            #imprime se o número de campos for diferente de 3
            if len(fields) != 3:
                print(f"Formato inesperado na linha: {line}")
                print(f"Campos extraídos: {fields}")
                
            # Se os campos forem válidos (3 itens), prossegue com o processamento
            if len(fields) == 3:
                action, URL, protocol = fields
                print(f"Ação: {action}, URL: {URL}, Protocolo: {protocol}")



['_\\xb0ZP\\x07tR\\xe5']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


Huh. In addition to empty fields, there's one that just contains garbage. Well, let's modify our script to check for that case:

In [21]:
#os 2 primeiros mais acessados fazia parte de um dado malicioso

URLCounts = {}  

with open(logPath, "r") as f:  
    for line in (l.rstrip() for l in f): 
        match = format_pat.match(line)  #
        if match:  #
            access = match.groupdict()  
            request = access['request']  # pega a string da solicitação 
            fields = request.split()  # Divide a string em uma lista com base em espaços
            if len(fields) == 3:  # Verifica se a solicitação contém exatamente 3 partes
                URL = fields[1]  # Extrai a URL (o segundo campo da solicitação)
                if URL in URLCounts:  #verifica se a URL ja esta no dicionario
                    URLCounts[URL] = URLCounts[URL] + 1  # Incrementa a contagem da URL
                else:
                    URLCounts[URL] = 1  # Adiciona a URL ao dicionario com contagem inicial de 1

# Ordena as URLs com base na contagem de acessos (em ordem decrescente)
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

for result in results[:20]:  # Itera sobre as 20 URLs mais acessadas
    print(result + ": " + str(URLCounts[result]))  # Exibe a URL e sua contagem de acessos


/xmlrpc.php: 68494
/wp-login.php: 1923
/: 440
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/business/: 70
/national/: 70
/national-headlines/: 70
/world/: 70
/weather/: 70


It worked! But, the results don't really make sense. What we really want is pages accessed by real humans looking for news from our little news site. What the heck is xmlrpc.php? A look at the log itself turns up a lot of entries like this:

46.166.139.20 - - [05/Dec/2015:05:19:35 +0000] "POST /xmlrpc.php HTTP/1.0" 200 370 "-" "Mozilla/4.0 (compatible: MSIE 7.0; Windows NT 6.0)"

I'm not entirely sure what the script does, but it points out that we're not just processing GET actions. We don't want POSTS, so let's filter those out:

In [22]:
#Com o get, comecou a ficar mais claro, mas ainda nao passa na verificacao, continua meio suspeito

URLCounts = {}  

with open(logPath, "r") as f:  
    for line in (l.rstrip() for l in f):  
        match = format_pat.match(line) 
        if match:  
            access = match.groupdict()  
            request = access['request']  # Obtem o valor da chave request que contém a solicitação
            fields = request.split()  
            if len(fields) == 3:  # Verifica se a solicitação tem exatamente 3 partes
                (action, URL, protocol) = fields  # Desempacota a solicitação para obter ação, URL e protocolo
                if action == 'GET':  # Verifica se a ação da solicitacao e GET
                    if URL in URLCounts:  
                        URLCounts[URL] = URLCounts[URL] + 1  # Incrementa o contador para a URL
                    else:
                        URLCounts[URL] = 1  # Se nao estiver, adiciona a URL ao dicionario com contador 1

# Ordena as URLs com base na contagem, em ordem decrescente
results = sorted(URLCounts, key=lambda i: int(URLCounts[i]), reverse=True)

# Exibe as 20 URLs mais acessadas
for result in results[:20]:  # Itera pelas 20 URLs mais acessadas
    print(result + ": " + str(URLCounts[result]))  # Exibe a URL e a contagem de acessos


/: 434
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/business/: 70
/national/: 70
/national-headlines/: 70
/world/: 70
/weather/: 70
/about/: 69
/defense-sticking-head-sand/: 69


That's starting to look better. But, this is a news site - are people really reading the little blog on it instead of news pages? That doesn't make sense. Let's look at a typical /blog/ entry in the log:

54.165.199.171 - - [05/Dec/2015:09:32:05 +0000] "GET /blog/ HTTP/1.0" 200 31670 "-" "-"

Hm. Why is the user agent blank? Seems like some sort of malicious scraper or something. Let's figure out what user agents we are dealing with:

In [31]:
#suspeito pois, os primeiros nao contem um formato de navegador comum, e sim de uma suposta invasao

UserAgents = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f): 
        match = format_pat.match(line)  # Tenta casar a linha com o formato de regex
        if match:  # Se a linha deu match
            access = match.groupdict()  #extrai os grupos nomeados da linha
            agent = access['user_agent']  # Extrai o user_agent do acesso
            if agent in UserAgents:  # Se o agente já foi registrado
                UserAgents[agent] = UserAgents[agent] + 1  #incrementa o contador de acessos
            else:
                UserAgents[agent] = 1  # Se for a primeira vez, inicializa o contador com 1

#ordena o dicionário UserAgents baseado nos valores de contagem
results = sorted(UserAgents.items(), key=lambda x: x[1], reverse=True)

# Exibe os 20 User-Agents mais acessados
for result in results[:20]:  # Limita a exibição aos 20 primeiros
    print(result[0] + ": " + str(result[1]))  # Exibe o User-Agent e a contagem de acessos


Mozilla/4.0 (compatible: MSIE 7.0; Windows NT 6.0): 68484
-: 4035
Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.0): 1724
W3 Total Cache/0.9.4.1: 468
Mozilla/5.0 (compatible; Baiduspider/2.0; +http://www.baidu.com/search/spider.html): 278
Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html): 248
Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36: 158
Mozilla/5.0 (Windows NT 6.1; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0: 144
Mozilla/5.0 (iPad; CPU OS 8_4 like Mac OS X) AppleWebKit/600.1.4 (KHTML, like Gecko) Version/8.0 Mobile/12H143 Safari/600.1.4: 120
Mozilla/5.0 (Linux; Android 5.1.1; SM-G900T Build/LMY47X) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.76 Mobile Safari/537.36: 47
Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm): 43
Mozilla/5.0 (compatible; MJ12bot/v1.4.5; http://www.majestic12.co.uk/bot.php?+): 41
Opera/9.80 (Windows NT 6.0) Presto/2.12.388 Version/12.1

Yikes! In addition to '-', there are also a million different web robots accessing the site and polluting my data. Filtering out all of them is really hard, but getting rid of the ones significantly polluting my data in this case should be a matter of getting rid of '-', anything containing "bot" or "spider", and W3 Total Cache.

In [27]:
URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match = format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']
            
            # ignora bots, spiders e outros agentes irrelevantes
            if (not ('bot' in agent or 'spider' in agent or
                     'Bot' in agent or 'Spider' in agent or
                     'W3 Total Cache' in agent or agent == '-')):
                
                request = access['request']
                fields = request.split()
                if len(fields) == 3:
                    action, URL, protocol = fields
                    if action == 'GET':  # So conta requisicoes GET
                        if URL in URLCounts:
                            URLCounts[URL] += 1  # incrementa a contagem de acessos
                        else:
                            URLCounts[URL] = 1  # inicializa a co|ntagem para a URL

# ordena o dicionário URLCounts com base nos valores de contagem 
results = sorted(URLCounts.items(), key=lambda x: x[1], reverse=True)

# Exibe as 20 URLs mais acessadas
for result in results[:20]:
    print(result[0] + ": " + str(result[1]))


/: 77
/orlando-headlines/: 36
/?page_id=34248: 28
/wp-content/cache/minify/000000/M9AvyUjVzUstLy7PLErVz8lMKkosqtTPKtYvTi7KLCgpBgA.js: 27
/wp-content/cache/minify/000000/M9bPKixNLarUy00szs8D0Zl5AA.js: 27
/wp-content/cache/minify/000000/lY7dDoIwDIVfiG0KxkfxfnbdKO4HuxICTy-it8Zw15PzfSftzPCckJem-x4qUWArqBPl5mygZLEgyhdOaoxToGyGaiALiOfUnIz0qDLOdSZGE-nOlpc3kopDzrSyavVVt_veb5qSDVhjsQ6dHh_B_eE_z2pYIGJ7iBWKeEio_eT9UQe4xHhDll27mGRryVu_pRc.js: 27
/wp-content/cache/minify/000000/fY45DoAwDAQ_FMvkRQgFA5ZyWLajiN9zNHR0O83MRkyt-pIctqYFJPedKyYzfHg2PzOFiENAzaD07AxcpKmTolORvDjZt8KEfhBUGjZYCf8Fb0fvA1TXCw.css: 25
/?author=1: 21
/wp-content/cache/minify/000000/hcrRCYAwDAXAhXyEjiQ1YKAh4SVSx3cE7_uG7ASr4M9qg3kGWyk1adklK84LHtRj_My6Y0Pfqcz-AA.js: 20
/wp-content/uploads/2014/11/nhn1.png: 19
/wp-includes/js/wp-emoji-release.min.js?ver=4.3.1: 17
/wp-content/cache/minify/000000/BcGBCQAgCATAiUSaKYSERPk3avzuht4SkBJnt4tHJdqgnPBqKldesTcN1R8.js: 17
/wp-login.php: 16
/comics-2/: 12
/world/: 12
/favicon.ico: 10
/wp-content/up

Now, our new problem is that we're getting a bunch of hits on things that aren't web pages. We're not interested in those, so let's filter out any URL that doesn't end in / (all of the pages on my site are accessed in that manner - again this is applying knowledge about my data to the analysis!)

In [32]:
#apos verificar se termina com "/", conseguimos resultados mais solidos, mais filtrados

URLCounts = {}

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):
        match = format_pat.match(line)
        if match:
            access = match.groupdict()
            agent = access['user_agent']
            if not ('bot' in agent or 'spider' in agent or 
                    'Bot' in agent or 'Spider' in agent or
                    'W3 Total Cache' in agent or agent == '-'):
                request = access['request']
                fields = request.split()
                if len(fields) == 3:
                    action, URL, protocol = fields
                    if URL.endswith("/"):  # Verifica se a URL termina com "/"
                        if action == 'GET':  # apenas conta as requisições GET
                            if URL in URLCounts:
                                URLCounts[URL] += 1  # incrementa a contagem de acessos
                            else:
                                URLCounts[URL] = 1  # inicializa a contagem para a URL

# ordena as URLs com base na contagem de acessos em ordem decrescente
results = sorted(URLCounts.items(), key=lambda x: x[1], reverse=True)

# exibe as 20 URLs mais acessadas
for result in results[:20]:
    print(result[0] + ": " + str(result[1]))


/: 77
/orlando-headlines/: 36
/comics-2/: 12
/world/: 12
/weather/: 4
/australia/: 4
/about/: 4
/national-headlines/: 3
/feed/: 2
/sample-page/feed/: 2
/science/: 2
/technology/: 2
/entertainment/: 1
/san-jose-headlines/: 1
/business/: 1
/travel/feed/: 1


This is starting to look more believable! But if you were to dig even deeper, you'd find that the /feed/ pages are suspect, and some robots are still slipping through. However, it is accurate to say that Orlando news, world news, and comics are the most popular pages accessed by a real human on this day.

The moral of the story is - know your data! And always question and scrutinize your results before making decisions based on them. If your business makes a bad decision because you provided an analysis of bad source data, you could get into real trouble.

Be sure the decisions you make while cleaning your data are justifiable too - don't strip out data just because it doesn't support the results you want!

## Activity

These results still aren't perfect; URL's that include "feed" aren't actually pages viewed by humans. Modify this code further to strip out URL's that include "/feed". Even better, extract some log entries for these pages and understand where these views are coming from.

In [34]:
#armazenando cagora com o feed, conseguimos visualizacoes mais precisas e da para visualizar de onde
#esta vindo as visualizacoes 

URLCounts = {}
FeedRequests = []  # Lista para armazenar linhas do log com "/feed" para análise

with open(logPath, "r") as f:
    for line in (l.rstrip() for l in f):  # Lê o arquivo linha por linha, removendo espaços em branco à direita
        match = format_pat.match(line)  # Verifica se a linha corresponde ao padrão esperado
        if match:
            access = match.groupdict()  # Extrai os campos nomeados do padrão
            request = access['request']  # Obtém o campo 'request'
            fields = request.split()  # Divide a string 'request' em partes

            # Verifica se o formato do request é válido e igual a 3 partes
            if len(fields) == 3:
                action, URL, protocol = fields  # Desempacota as partes em variáveis

                # Filtra URLs que contenham "/feed"
                if "/feed" in URL:
                    FeedRequests.append(line)  # Adiciona a linha do log à lista para análise
                    continue  # Pula a contagem para essas URLs
                
                # Apenas conta requests do tipo GET
                if action == "GET":
                    if URL in URLCounts:
                        URLCounts[URL] += 1  # Incrementa a contagem para a URL ja existente
                    else:
                        URLCounts[URL] = 1  # Inicializa a contagem para uma nova URL

# Ordena os resultados com base na contagem das URLs
results = sorted(URLCounts, key=lambda i: URLCounts[i], reverse=True)

# Exibe as 20 URLs mais acessadas
print("Top 20 URLs:")
for result in results[:20]:
    print(result + ": " + str(URLCounts[result]))

# Analisa as entradas relacionadas a "/feed"
print("\nEntradas do log para '/feed':")
for feed_request in FeedRequests[:10]:  # Exibe as primeiras 10 entradas para análise
    print(feed_request)


Top 20 URLs:
/: 434
/blog/: 138
/robots.txt: 123
/sitemap_index.xml: 118
/post-sitemap.xml: 118
/page-sitemap.xml: 117
/category-sitemap.xml: 117
/orlando-headlines/: 95
/san-jose-headlines/: 85
http://51.254.206.142/httptest.php: 81
/comics-2/: 76
/travel/: 74
/entertainment/: 72
/business/: 70
/national/: 70
/national-headlines/: 70
/world/: 70
/weather/: 70
/about/: 69
/defense-sticking-head-sand/: 69

Entradas do log para '/feed':
195.154.250.88 - - [29/Nov/2015:22:35:35 +0000] "GET /feed/ HTTP/1.1" 200 17592 "http://www.nohatenews.com/?feed=rss2" "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.0)"
46.166.139.20 - - [29/Nov/2015:22:39:44 +0000] "GET /feed/ HTTP/1.1" 200 17592 "-" "-"
46.165.197.141 - - [29/Nov/2015:23:04:39 +0000] "GET /washington-dc-sports/feed/ HTTP/1.0" 200 578 "-" "Mozilla/5.0 (compatible; MJ12bot/v1.4.5; http://www.majestic12.co.uk/bot.php?+)"
46.4.89.35 - - [30/Nov/2015:13:18:21 +0000] "GET /about/feed/ HTTP/1.0" 200 994 "-" "Mozilla/5.0 (compatible; MJ12bot